<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [3]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [4]:

df = pd.DataFrame()
dfs = []
for f in archivos_anio:
    tmp = pd.read_csv(f)
    tmp.columns = tmp.columns.str.lower()
    dfs.append(tmp)

df_anio = pd.concat(dfs, ignore_index=True)

df_codigos['codigo_iso'].value_counts()

df_codigos = df_codigos.drop_duplicates(subset=['codigo_iso'], keep='first')

df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')
df.head()

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [9]:
df.info()

df.groupby('codigo_iso')['pais'].nunique().value_counts()
print("Estructura del df")
print("Filas y columnas:", df.shape)
print("Nombres de columnas:", list(df.columns))
print("\nTipos de datos:")
print(df.dtypes)

print("\n Resumen estadistico")
print(df.describe())

print("\nIndice minimo:", df['indice'].min(), " ", df.loc[df['indice'].idxmin(), 'pais'])
print("Indice maximo:", df['indice'].max(), " ", df.loc[df['indice'].idxmax(), 'pais'])
print("Ranking mejor (minimo):", df['ranking'].min(), " ", df.loc[df['ranking'].idxmin(), 'pais'])
print("Ranking peor (maximo):", df['ranking'].max(), " ", df.loc[df['ranking'].idxmax(), 'pais'])

print("\n Datos faltantes")
print(df.isnull().sum())
print("Proporcion de los nulos por columna:\n", df.isnull().mean())

print("Unicidad y duplicad")
print("Paises distintos:", df['pais'].nunique())
print("Años distintos:", df['anio'].nunique())
print("Filas duplicadas:", df.duplicated().sum())

print(df.groupby('codigo_iso')['pais'].nunique().value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  3060 non-null   object 
 1   anio        3060 non-null   int64  
 2   indice      2664 non-null   float64
 3   ranking     2837 non-null   float64
 4   pais        3060 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 119.7+ KB
Estructura del df
Filas y columnas: (3060, 5)
Nombres de columnas: ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

Tipos de datos:
codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object

 Resumen estadistico
              anio        indice        ranking
count  3060.000000   2664.000000    2837.000000
mean   2009.941176    205.782316     477.930913
std       5.786024   2695.525264    6474.935347
min    2001.000000      0.000000       1.000000
25%    2005.000000   




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [21]:
america = ['ARG','ATG','BLZ','BOL','BRA','CAN','CHL','COL','CRI',
           'CUB','DOM','ECU','GRD','GTM','GUY','HND','HTI','JAM',
           'MEX','NIC','PAN','PER','PRY','SLV','SUR','TTO','URY',
           'USA','VEN']

df_america = df[df["codigo_iso"].isin(america)].copy()

anios = sorted(df_america["anio"].dropna().unique())

for anio in anios:
    datos_anio = df_america[df_america["anio"] == anio].dropna(subset=["indice"])
    if datos_anio.empty:
        print(f"Año {anio}: no hay datos de 'indice'")
        continue

    min_row = datos_anio.loc[datos_anio["indice"].idxmin()]
    max_row = datos_anio.loc[datos_anio["indice"].idxmax()]

    print(f"\nAño {anio}:")
    print(f"Mayor libertad de prensa: {min_row['pais']} (Índice = {min_row['indice']})")
    print(f"Menor libertad de prensa: {max_row['pais']} (Índice = {max_row['indice']})")
df_valid = df_america.dropna(subset=["indice"])
idx_min = df_valid.groupby("anio")["indice"].idxmin()
idx_max = df_valid.groupby("anio")["indice"].idxmax()
df_min = df_valid.loc[idx_min, ["anio", "pais", "indice"]].set_index("anio").rename(columns={"pais":"mayor_libertad", "indice":"indice_min"})
df_max = df_valid.loc[idx_max, ["anio", "pais", "indice"]].set_index("anio").rename(columns={"pais":"menor_libertad", "indice":"indice_max"})

resultados = pd.concat([df_min, df_max], axis=1).reset_index()

resultados = resultados[["anio", "mayor_libertad", "indice_min", "menor_libertad", "indice_max"]]

print(resultados)


Año 2001:
  Mayor libertad de prensa: Canadá (Índice = 0.8)
  Menor libertad de prensa: Cuba (Índice = 90.3)

Año 2002:
  Mayor libertad de prensa: Trinidad y Tobago (Índice = 1.0)
  Menor libertad de prensa: Cuba (Índice = 97.83)

Año 2003:
  Mayor libertad de prensa: Trinidad y Tobago (Índice = 2.0)
  Menor libertad de prensa: Argentina (Índice = 35826.0)

Año 2004:
  Mayor libertad de prensa: Trinidad y Tobago (Índice = 2.0)
  Menor libertad de prensa: Cuba (Índice = 87.0)

Año 2005:
  Mayor libertad de prensa: Bolivia (Índice = 4.5)
  Menor libertad de prensa: Cuba (Índice = 95.0)

Año 2006:
  Mayor libertad de prensa: Canadá (Índice = 4.88)
  Menor libertad de prensa: Cuba (Índice = 96.17)

Año 2007:
  Mayor libertad de prensa: Canadá (Índice = 3.33)
  Menor libertad de prensa: Cuba (Índice = 88.33)

Año 2008:
  Mayor libertad de prensa: Canadá (Índice = 3.7)
  Menor libertad de prensa: Cuba (Índice = 94.0)

Año 2009:
  Mayor libertad de prensa: Estados Unidos (Índice = 6.75)
  M

### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [22]:
tabla = pd.pivot_table(
    df_america,
    values="indice",
    index="pais",
    columns="anio",
    aggfunc="max",
    fill_value=0)

print(tabla)

max_val = tabla.max().max()
min_val = tabla[tabla > 0].min().min()

pais_max = tabla.where(tabla == max_val).stack().index[0][0]
pais_min = tabla.where(tabla == min_val).stack().index[0][0]

print("\n(a)")
print(f"Pais con mayor indice: {pais_max} ({max_val})")
print(f"Pais con menor indice (distinto de 0): {pais_min} ({min_val})")

promedios_anio = tabla.mean(axis=0)
anio_alto = promedios_anio.idxmax()
anio_bajo = promedios_anio.idxmin()

print("\n(b)")
print(f"Anio con promedio mas alto: {anio_alto} ({promedios_anio.max()})")
print(f"Anio con promedio mas bajo: {anio_bajo} ({promedios_anio.min()})")

variabilidad = tabla.max(axis=1) - tabla.min(axis=1)
pais_mas_variable = variabilidad.idxmax()

print("\n(c)")
print(f"Pais con mayor variabilidad: {pais_mas_variable} ({variabilidad.max()})")

constantes = variabilidad[variabilidad == 0].index.tolist()

print("\n(d)")
print("Paises con indice constante:", constantes)

sin_datos = tabla[tabla.sum(axis=1) == 0].index.tolist()

print("\n(e)")
print("Paises sin ningun dato:", sin_datos)

anio                  2001   2002      2003   2004   2005   2006   2007  \
pais                                                                      
Antigua y Barbuda      0.0   0.00      0.00   0.00   0.00   0.00   0.00   
Argentina             12.0  15.17  35826.00  13.67  17.30  24.83  14.08   
Belize                 0.0   0.00      0.00   0.00   0.00   0.00   0.00   
Bolivia               14.5   9.67     20.00   9.67   4.50  21.50  28.20   
Brasil                18.8  16.75     16.50  14.50  17.17  25.25  18.00   
Canadá                 0.8   1.83      3.33   4.50   4.50   4.88   3.33   
Chile                  6.5   6.83     10.00  11.75  11.63  12.13  11.50   
Colombia              40.8  49.17     47.38  40.17  44.75  42.33  35.50   
Costa Rica             4.3   3.83      7.63   8.50   6.67   6.50   5.10   
Cuba                  90.3  97.83    106.83  87.00  95.00  96.17  88.33   
Ecuador                5.5   7.67     16.50  21.75  15.25  18.50  15.50   
El Salvador            8.